In [ ]:
import ee

ee.Authenticate()

ee.Initialize()

In [ ]:
# Taken from Earth Engine's guide

import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
geometry_1 = ee.Geometry.Polygon(
    [
        [28.790950060779327,-23.188126816120498],
        [29.197444201404327,-23.188126816120498],
        [29.197444201404327,-22.959446379546044],
        [28.790950060779327,-22.959446379546044],
    ]
)
geometry_2 = ee.Geometry.Polygon(
    [
        [29.311319978172413,-23.11302383297978],
        [29.654642732078663,-23.11302383297978],
        [29.654642732078663,-22.909517488129257],
        [29.311319978172413,-22.909517488129257],
    ]
)

l5_7_raw = ee.ImageCollection("LANDSAT/GLS2005")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA")
l7_2 = ee.ImageCollection("LANDSAT/LE07/C02/T2_TOA")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
l8_2 = ee.ImageCollection("LANDSAT/LC08/C02/T2_TOA")

In [7]:
# Getting the images
# Position: 29.258, -22.997021
years = {
    "1999": (l7, "B3", "B4"),
    "2002": (l7_2, "B3", "B4"),
    "2005": (l7, "B3", "B4"),
    "2006": (l5_7_raw, "30", "40"),
    "2008": (l7, "B3", "B4"),
    "2014": (l8, "B4", "B5"),
    "2017": (l8, "B4", "B5"),
    "2020": (l8, "B4", "B5"),
}

images = []

geometry = geometry_1

for year, dataset in years.items():
    image = ee.Image(
        dataset[0].filterBounds(geometry)
        .filterDate(f"{year}-01-01", f"{year}-12-31")
        .sort("CLOUD_COVER")
        .first()
    ).clip(geometry)
    images.append(image)

print(images)

[<ee.image.Image object at 0x000001E669867F50>, <ee.image.Image object at 0x000001E669E16C10>, <ee.image.Image object at 0x000001E669E17A10>, <ee.image.Image object at 0x000001E669DF3710>, <ee.image.Image object at 0x000001E669E1FA10>, <ee.image.Image object at 0x000001E66A68AA90>, <ee.image.Image object at 0x000001E66A6B0510>, <ee.image.Image object at 0x000001E66A6B2250>]


In [8]:
import numpy as np
import os
import geemap

# Getting the red band and near infrared band
for year, dataset, image in zip(years.keys(), years.values(), images):
    red_band_name, nir_band_name = dataset[1:]

    red_band = image.select(red_band_name)
    nir_band = image.select(nir_band_name)

    # Change between directory 1 and 2
    out_dir = os.path.join("images_1")

    red_filename = os.path.join(out_dir, f'{year}_red.tif')
    geemap.ee_export_image(
        red_band, filename=red_filename
    )
    nir_filename = os.path.join(out_dir, f'{year}_nir.tif')
    geemap.ee_export_image(
        nir_band, filename=nir_filename
    )


Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\1999_red.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\1999_nir.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\2002_red.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\2002_nir.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\2005_red.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\2005_nir.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\Documents\Coding\scuola\astropi-2223\ndvi\images_2\2006_red.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gabri\

In [13]:
import ndvi
import csv
# Analysis

values = [("Year", "Average NDVI")]


def add_values(arr, year):
    val = year, np.average(arr)#, np.max(arr), np.min(arr)
    values.append(val)


for year in years.keys():
    # Change between directory 1 and 2
    directory = "converted_images_1"
    calculated_ndvi = ndvi.two_image_ndvi_from_tiff(
        os.path.join(directory, f"{year}_red.png"),
        os.path.join(directory, f"{year}_nir.png"),
    )
    add_values(calculated_ndvi, year)

ndvi_2023 = ndvi.NDVI(os.path.join(directory, "2023.png"))

add_values(
    ndvi_2023.ndvi, "2023"
)

with open("ndvi_values.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(values)

In [ ]:
# Visualization

lat, lon = -22.9970214098314, 29.258209136397067
my_map = folium.Map(location=[lat, lon], zoom_start=9)

parameters = {"min": -1, "max": 1, "palette": ["white", "black"]}

my_map.add_ee_layer(red_band, parameters, "NDVI")

my_map